In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
print os.listdir('drive')

# import sys
# sys.path.append('drive/places_fc2')
# sys.path.append('drive/cnn')

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
['.Trash', 'siamese', 'tfsiamese', 'capgen', 'imgcap', 'Colab Notebooks', 'GupShup', 'Music Recommender', 'Distilled AI', 'springerformat.doc', 'Untitled document.odt', 'Computer Graphics', 'ComputerGraphics_Programs.zip', 'Friends Season 6', 'F.R.I.E.N.D.S season -5', 'Friends Season 4', 'Copy of Dataset.tar.

In [2]:
!pip install keras

    100% |████████████████████████████████| 327kB 1.8MB/s 


In [3]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                   )
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    model.layers.pop() # Get rid of the classification layer
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    return model


Using TensorFlow backend.


In [0]:
from keras.applications import inception_v3
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint
import cPickle as pickle

EMBEDDING_DIM = 128


class CaptionGenerator():

    def __init__(self):
        self.max_cap_len = None
        self.vocab_size = None
        self.index_word = None
        self.word_index = None
        self.total_samples = None
        self.encoded_images = pickle.load( open( "drive/capgen/encoded_images.p", "rb" ) )
        self.variable_initializer()

    def variable_initializer(self):
        df = pd.read_csv('drive/capgen/Flickr8k_text/flickr_8k_train_dataset.txt', delimiter='\t')
        nb_samples = df.shape[0]
        iter = df.iterrows()
        caps = []
        for i in range(nb_samples):
            x = iter.next()
            caps.append(x[1][1])

        self.total_samples=0
        for text in caps:
            self.total_samples+=len(text.split())-1
        print ("Total samples : "+str(self.total_samples))
        
        words = [txt.split() for txt in caps]
        unique = []
        for word in words:
            unique.extend(word)

        unique = list(set(unique))
        self.vocab_size = len(unique)
        self.word_index = {}
        self.index_word = {}
        for i, word in enumerate(unique):
            self.word_index[word]=i
            self.index_word[i]=word

        max_len = 0
        for caption in caps:
            if(len(caption.split()) > max_len):
                max_len = len(caption.split())
        self.max_cap_len = max_len
        print ("Vocabulary size: "+str(self.vocab_size))
        print ("Maximum caption length: "+str(self.max_cap_len))
        print ("Variables initialization done!")


    def data_generator(self, batch_size = 32):
        partial_caps = []
        next_words = []
        images = []
        print ("Generating data...")
        gen_count = 0
        df = pd.read_csv('drive/capgen/Flickr8k_text/flickr_8k_train_dataset.txt', delimiter='\t')
        nb_samples = df.shape[0]
        iter = df.iterrows()
        caps = []
        imgs = []
        for i in range(nb_samples):
            x = iter.next()
            caps.append(x[1][1])
            imgs.append(x[1][0])


        total_count = 0
        while 1:
            image_counter = -1
            for text in caps:
                image_counter+=1
                current_image = self.encoded_images[imgs[image_counter]]
                for i in range(len(text.split())-1):
                    total_count+=1
                    partial = [self.word_index[txt] for txt in text.split()[:i+1]]
                    partial_caps.append(partial)
                    next = np.zeros(self.vocab_size)
                    next[self.word_index[text.split()[i+1]]] = 1
                    next_words.append(next)
                    images.append(current_image)

                    if total_count>=batch_size:
                        next_words = np.asarray(next_words)
                        images = np.asarray(images)
                        partial_caps = sequence.pad_sequences(partial_caps, maxlen=self.max_cap_len, padding='post')
                        total_count = 0
                        gen_count+=1
                        print ("yielding count: "+str(gen_count))
                        yield [[images, partial_caps], next_words]
                        partial_caps = []
                        next_words = []
                        images = []
        
    def load_image(self, path):
        img = image.load_img(path, target_size=(224,224))
        x = image.img_to_array(img)
        return np.asarray(x)


    def create_model(self, ret_model = False):
        #base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
        #base_model.trainable=False
        image_model = Sequential()
        #image_model.add(base_model)
        #image_model.add(Flatten())
        image_model.add(Dense(EMBEDDING_DIM, input_dim = 4096, activation='relu'))

        image_model.add(RepeatVector(self.max_cap_len))

        lang_model = Sequential()
        lang_model.add(Embedding(self.vocab_size, 256, input_length=self.max_cap_len))
        lang_model.add(LSTM(256,return_sequences=True))
        lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))

        model = Sequential()
        model.add(Merge([image_model, lang_model], mode='concat'))
        model.add(LSTM(1000,return_sequences=False))
        model.add(Dense(self.vocab_size))
        model.add(Activation('softmax'))

        print ("Model created!")

        if(ret_model==True):
            return model

        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        return model

    def get_word(self,index):
        return self.index_word[index]

In [0]:
from keras.callbacks import ModelCheckpoint

def train_model(weight = None, batch_size=32, epochs = 10):

    cg = CaptionGenerator()
    model = cg.create_model()

    model.load_weights('drive/capgen/weights-improvement1-01.hdf5')

    counter = 0
    file_name = 'drive/capgen/weights-improvement2-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit_generator(cg.data_generator(batch_size=batch_size), steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=2, callbacks=callbacks_list)
    try:
        model.save('drive/capgen/Models/WholeModel.h5', overwrite=True)
        model.save_weights('drive/capgen/Models/Weights.h5',overwrite=True)
    except:
        print ("Error in sav98++ing model.")
    print ("Training complete...\n")

In [6]:
train_model(epochs=1)

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Epoch 1/1
Generating data...
yielding count: 1
yielding count: 2
yielding count: 3
yielding count: 4
yielding count: 5
yielding count: 6
yielding count: 7
yielding count: 8
yielding count: 9
yielding count: 10
yielding count: 11
yielding count: 12
yielding count: 13
yielding count: 14
yielding count: 15
yielding count: 16
yielding count: 17
yielding count: 18
yielding count: 19
yielding count: 20
yielding count: 21
yielding count: 22
yielding count: 23
yielding count: 24
yielding count: 25
yielding count: 26
yielding count: 27
yielding count: 28
yielding count: 29
yielding count: 30
yielding count: 31
yielding count: 32
yielding count: 33
yielding count: 34
yielding count: 35
yielding count: 36
yielding count: 37
yielding count: 38
yielding count: 39
yielding count: 40
yielding count: 41
yielding count: 42
yielding count: 43
yielding count: 44
yielding count: 45
yielding count: 46
yielding count: 47
yielding count: 48
yielding count: 49
yielding count: 50
yielding count:

yielding count: 101
yielding count: 102
yielding count: 103
yielding count: 104
yielding count: 105
yielding count: 106
yielding count: 107
yielding count: 108
yielding count: 109
yielding count: 110
yielding count: 111
yielding count: 112
yielding count: 113
yielding count: 114
yielding count: 115
yielding count: 116
yielding count: 117
yielding count: 118
yielding count: 119
yielding count: 120
yielding count: 121
yielding count: 122
yielding count: 123
yielding count: 124
yielding count: 125
yielding count: 126
yielding count: 127
yielding count: 128
yielding count: 129
yielding count: 130
yielding count: 131
yielding count: 132
yielding count: 133
yielding count: 134
yielding count: 135
yielding count: 136
yielding count: 137
yielding count: 138
yielding count: 139
yielding count: 140
yielding count: 141
yielding count: 142
yielding count: 143
yielding count: 144
yielding count: 145
yielding count: 146
yielding count: 147
yielding count: 148
yielding count: 149
yielding count: 150


yielding count: 200
yielding count: 201
yielding count: 202
yielding count: 203
yielding count: 204
yielding count: 205
yielding count: 206
yielding count: 207
yielding count: 208
yielding count: 209
yielding count: 210
yielding count: 211
yielding count: 212
yielding count: 213
yielding count: 214
yielding count: 215
yielding count: 216
yielding count: 217
yielding count: 218
yielding count: 219
yielding count: 220
yielding count: 221
yielding count: 222
yielding count: 223
yielding count: 224
yielding count: 225
yielding count: 226
yielding count: 227
yielding count: 228
yielding count: 229
yielding count: 230
yielding count: 231
yielding count: 232
yielding count: 233
yielding count: 234
yielding count: 235
yielding count: 236
yielding count: 237
yielding count: 238
yielding count: 239
yielding count: 240
yielding count: 241
yielding count: 242
yielding count: 243
yielding count: 244
yielding count: 245
yielding count: 246
yielding count: 247
yielding count: 248
yielding count: 249


...

yielding count: 11611
yielding count: 11612
yielding count: 11613
yielding count: 11614
yielding count: 11615
yielding count: 11616
yielding count: 11617
yielding count: 11618
yielding count: 11619
yielding count: 11620
yielding count: 11621
yielding count: 11622
yielding count: 11623
yielding count: 11624
yielding count: 11625
yielding count: 11626
yielding count: 11627
yielding count: 11628
yielding count: 11629
yielding count: 11630
yielding count: 11631
yielding count: 11632
yielding count: 11633
yielding count: 11634
yielding count: 11635
yielding count: 11636
yielding count: 11637
yielding count: 11638
yielding count: 11639
yielding count: 11640
yielding count: 11641
yielding count: 11642
yielding count: 11643
yielding count: 11644
yielding count: 11645
yielding count: 11646
yielding count: 11647
yielding count: 11648
yielding count: 11649
yielding count: 11650
yielding count: 11651
yielding count: 11652
yielding count: 11653
yielding count: 11654
yielding count: 11655
yielding c

yielding count: 11705
yielding count: 11706
yielding count: 11707
yielding count: 11708
yielding count: 11709
yielding count: 11710
yielding count: 11711
yielding count: 11712
yielding count: 11713
yielding count: 11714
yielding count: 11715
yielding count: 11716
yielding count: 11717
yielding count: 11718
yielding count: 11719
yielding count: 11720
yielding count: 11721
yielding count: 11722
yielding count: 11723
yielding count: 11724
yielding count: 11725
yielding count: 11726
yielding count: 11727
yielding count: 11728
yielding count: 11729
yielding count: 11730
yielding count: 11731
yielding count: 11732
yielding count: 11733
yielding count: 11734
yielding count: 11735
yielding count: 11736
yielding count: 11737
yielding count: 11738
yielding count: 11739
yielding count: 11740
yielding count: 11741
yielding count: 11742
yielding count: 11743
yielding count: 11744
yielding count: 11745
yielding count: 11746
yielding count: 11747
yielding count: 11748
yielding count: 11749
yielding c

yielding count: 11799
yielding count: 11800
yielding count: 11801
yielding count: 11802
yielding count: 11803
yielding count: 11804
yielding count: 11805
yielding count: 11806
yielding count: 11807
yielding count: 11808
yielding count: 11809
yielding count: 11810
yielding count: 11811
yielding count: 11812
yielding count: 11813
yielding count: 11814
yielding count: 11815
yielding count: 11816
yielding count: 11817
yielding count: 11818
yielding count: 11819
yielding count: 11820
yielding count: 11821
yielding count: 11822
yielding count: 11823
yielding count: 11824
yielding count: 11825
yielding count: 11826
yielding count: 11827
yielding count: 11828
yielding count: 11829
yielding count: 11830
yielding count: 11831
yielding count: 11832
yielding count: 11833
yielding count: 11834
yielding count: 11835
yielding count: 11836
yielding count: 11837
yielding count: 11838
yielding count: 11839
yielding count: 11840
yielding count: 11841
yielding count: 11842
yielding count: 11843
yielding c

yielding count: 11893
yielding count: 11894
yielding count: 11895
yielding count: 11896
yielding count: 11897
yielding count: 11898
yielding count: 11899
yielding count: 11900
yielding count: 11901
yielding count: 11902
yielding count: 11903
yielding count: 11904
yielding count: 11905
yielding count: 11906
yielding count: 11907
yielding count: 11908
yielding count: 11909
yielding count: 11910
yielding count: 11911
yielding count: 11912
yielding count: 11913
yielding count: 11914
yielding count: 11915
yielding count: 11916
yielding count: 11917
yielding count: 11918
yielding count: 11919
yielding count: 11920
yielding count: 11921
yielding count: 11922
yielding count: 11923
yielding count: 11924
yielding count: 11925
yielding count: 11926
yielding count: 11927
yielding count: 11928
yielding count: 11929
yielding count: 11930
yielding count: 11931
yielding count: 11932
yielding count: 11933
yielding count: 11934
yielding count: 11935
yielding count: 11936
yielding count: 11937
yielding c

yielding count: 11987
yielding count: 11988
yielding count: 11989
yielding count: 11990
yielding count: 11991
yielding count: 11992
 - 2858s - loss: 4.9209 - acc: 0.3291

Epoch 00001: loss improved from inf to 4.92092, saving model to drive/capgen/weights-improvement2-01.hdf5
Training complete...



In [0]:
from keras.callbacks import ModelCheckpoint

def train_model(weight = None, batch_size=32, epochs = 10):

    cg = CaptionGenerator()
    model = cg.create_model()

    model.load_weights('drive/capgen/weights-improvement2-01.hdf5')

    counter = 0
    file_name = 'drive/capgen/weights-improvement3-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit_generator(cg.data_generator(batch_size=batch_size), steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=2, callbacks=callbacks_list)
    try:
        model.save('drive/capgen/Models/WholeModel2.h5', overwrite=True)
        model.save_weights('drive/capgen/Models/Weights2.h5',overwrite=True)
    except:
        print ("Error in saving model.")
    print ("Training complete...\n")

In [0]:
train_model(epochs=1)

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Variables initialization done!


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:127: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Model created!
Generating data...Epoch 1/1

yielding count: 1
yielding count: 2
yielding count: 3
yielding count: 4
yielding count: 5
yielding count: 6
yielding count: 7
yielding count: 8
yielding count: 9
yielding count: 10
yielding count: 11
yielding count: 12
yielding count: 13
yielding count: 14
yielding count: 15
yielding count: 16
yielding count: 17
yielding count: 18
yielding count: 19
yielding count: 20
yielding count: 21
yielding count: 22
yielding count: 23
yielding count: 24
yielding count: 25
yielding count: 26
yielding count: 27
yielding count: 28
yielding count: 29
yielding count: 30
yielding count: 31
yielding count: 32
yielding count: 33
yielding count: 34
yielding count: 35
yielding count: 36
yielding count: 37
yielding count: 38
yielding count: 39
yielding count: 40
yielding count: 41
yielding count: 42
yielding count: 43
yielding count: 44
yielding count: 45
yielding count: 46
yielding count: 47
yielding count: 48
yielding count: 49
yielding count: 50
yielding count:

yielding count: 101
yielding count: 102
yielding count: 103
yielding count: 104
yielding count: 105
yielding count: 106
yielding count: 107
yielding count: 108
yielding count: 109
yielding count: 110
yielding count: 111
yielding count: 112
yielding count: 113
yielding count: 114
yielding count: 115
yielding count: 116
yielding count: 117
yielding count: 118
yielding count: 119
yielding count: 120
yielding count: 121
yielding count: 122
yielding count: 123
yielding count: 124
yielding count: 125
yielding count: 126
yielding count: 127
yielding count: 128
yielding count: 129
yielding count: 130
yielding count: 131
yielding count: 132
yielding count: 133
yielding count: 134
yielding count: 135
yielding count: 136
yielding count: 137
yielding count: 138
yielding count: 139
yielding count: 140
yielding count: 141
yielding count: 142
yielding count: 143
yielding count: 144
yielding count: 145
yielding count: 146
yielding count: 147
yielding count: 148
yielding count: 149
yielding count: 150


yielding count: 200
yielding count: 201
yielding count: 202
yielding count: 203
yielding count: 204
yielding count: 205
yielding count: 206
yielding count: 207
yielding count: 208
yielding count: 209
yielding count: 210
yielding count: 211
yielding count: 212
yielding count: 213
yielding count: 214
yielding count: 215
yielding count: 216
yielding count: 217
yielding count: 218
yielding count: 219
yielding count: 220
yielding count: 221
yielding count: 222
yielding count: 223
yielding count: 224
yielding count: 225
yielding count: 226
yielding count: 227
yielding count: 228
yielding count: 229
yielding count: 230
yielding count: 231
yielding count: 232
yielding count: 233
yielding count: 234
yielding count: 235
yielding count: 236
yielding count: 237
yielding count: 238
yielding count: 239
yielding count: 240
yielding count: 241
yielding count: 242
yielding count: 243
yielding count: 244
yielding count: 245
yielding count: 246
yielding count: 247
yielding count: 248
yielding count: 249


...

yielding count: 4873
yielding count: 4874
yielding count: 4875
yielding count: 4876
yielding count: 4877
yielding count: 4878
yielding count: 4879
yielding count: 4880
yielding count: 4881
yielding count: 4882
yielding count: 4883
yielding count: 4884
yielding count: 4885
yielding count: 4886
yielding count: 4887
yielding count: 4888
yielding count: 4889
yielding count: 4890
yielding count: 4891
yielding count: 4892
yielding count: 4893
yielding count: 4894
yielding count: 4895
yielding count: 4896
yielding count: 4897
yielding count: 4898
yielding count: 4899
yielding count: 4900
yielding count: 4901
yielding count: 4902
yielding count: 4903
yielding count: 4904
yielding count: 4905
yielding count: 4906
yielding count: 4907
yielding count: 4908
yielding count: 4909
yielding count: 4910
yielding count: 4911
yielding count: 4912
yielding count: 4913
yielding count: 4914
yielding count: 4915
yielding count: 4916
yielding count: 4917
yielding count: 4918
yielding count: 4919
yielding coun

yielding count: 4970
yielding count: 4971
yielding count: 4972
yielding count: 4973
yielding count: 4974
yielding count: 4975
yielding count: 4976
yielding count: 4977
yielding count: 4978
yielding count: 4979
yielding count: 4980
yielding count: 4981
yielding count: 4982
yielding count: 4983
yielding count: 4984
yielding count: 4985
yielding count: 4986
yielding count: 4987
yielding count: 4988
yielding count: 4989
yielding count: 4990
yielding count: 4991
yielding count: 4992
yielding count: 4993
yielding count: 4994
yielding count: 4995
yielding count: 4996
yielding count: 4997
yielding count: 4998
yielding count: 4999
yielding count: 5000
yielding count: 5001
yielding count: 5002
yielding count: 5003
yielding count: 5004
yielding count: 5005
yielding count: 5006
yielding count: 5007
yielding count: 5008
yielding count: 5009
yielding count: 5010
yielding count: 5011
yielding count: 5012
yielding count: 5013
yielding count: 5014
yielding count: 5015
yielding count: 5016
yielding coun

yielding count: 5067
yielding count: 5068
yielding count: 5069
yielding count: 5070
yielding count: 5071
yielding count: 5072
yielding count: 5073
yielding count: 5074
yielding count: 5075
yielding count: 5076
yielding count: 5077
yielding count: 5078
yielding count: 5079
yielding count: 5080
yielding count: 5081
yielding count: 5082
yielding count: 5083
yielding count: 5084
yielding count: 5085
yielding count: 5086
yielding count: 5087
yielding count: 5088
yielding count: 5089
yielding count: 5090
yielding count: 5091
yielding count: 5092
yielding count: 5093
yielding count: 5094
yielding count: 5095
yielding count: 5096
yielding count: 5097
yielding count: 5098
yielding count: 5099
yielding count: 5100
yielding count: 5101
yielding count: 5102
yielding count: 5103
yielding count: 5104
yielding count: 5105
yielding count: 5106
yielding count: 5107
yielding count: 5108
yielding count: 5109
yielding count: 5110
yielding count: 5111
yielding count: 5112
yielding count: 5113
yielding coun

yielding count: 5164
yielding count: 5165
yielding count: 5166
yielding count: 5167
yielding count: 5168
yielding count: 5169
yielding count: 5170
yielding count: 5171
yielding count: 5172
yielding count: 5173
yielding count: 5174
yielding count: 5175
yielding count: 5176
yielding count: 5177
yielding count: 5178
yielding count: 5179
yielding count: 5180
yielding count: 5181
yielding count: 5182
yielding count: 5183
yielding count: 5184
yielding count: 5185
yielding count: 5186
yielding count: 5187
yielding count: 5188
yielding count: 5189
yielding count: 5190
yielding count: 5191
yielding count: 5192
yielding count: 5193
yielding count: 5194
yielding count: 5195
yielding count: 5196
yielding count: 5197
yielding count: 5198
yielding count: 5199
yielding count: 5200
yielding count: 5201
yielding count: 5202
yielding count: 5203
yielding count: 5204
yielding count: 5205
yielding count: 5206
yielding count: 5207
yielding count: 5208
yielding count: 5209
yielding count: 5210
yielding coun

yielding count: 5261
yielding count: 5262
yielding count: 5263
yielding count: 5264
yielding count: 5265
yielding count: 5266
yielding count: 5267
yielding count: 5268
yielding count: 5269
yielding count: 5270
yielding count: 5271


In [0]:
from keras.callbacks import ModelCheckpoint

def train_model(weight = None, batch_size=32, epochs = 10):

    cg = CaptionGenerator()
    model = cg.create_model()

    model.load_weights('drive/capgen/weights-improvement3-01.hdf5')

    counter = 0
    file_name = 'drive/capgen/weights-improvement4-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit_generator(cg.data_generator(batch_size=batch_size), steps_per_epoch=cg.total_samples/batch_size, epochs=epochs, verbose=2, callbacks=callbacks_list)
    try:
        model.save('drive/capgen/Models/WholeModel3.h5', overwrite=True)
        model.save_weights('drive/capgen/Models/Weights3.h5',overwrite=True)
    except:
        print ("Error in saving model.")
    print ("Training complete...\n")

In [0]:
train_model(epochs=1)